In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve,auc
from scipy.stats import ks_2samp

In [2]:
df = pd.read_csv("dataset_w4.csv")

In [3]:
df['create_date'] = df['create_date'].astype('datetime64')

In [5]:
df['2017-create_date'] = df['create_date'].apply(lambda x: datetime.datetime.strptime('2017-01-01 00:00:00', '%Y-%m-%d %H:%M:%S') - x)
df['2017-create_date'] = (df['2017-create_date']/np.timedelta64(1, 'Y'))
df['2017-create_date'] = df['2017-create_date'][df['2017-create_date'].notnull()].astype("int")

In [16]:
sectors = list(set(df['Sector']))[1:]
writer = pd.ExcelWriter("summary.xlsx",engine='openpyxl')
for i in sectors:
    df1 = df[df['Sector'] == i][['dummy_1','dummy_2','dummy_3','dummy_4','dummy_5','dummy_6','media_pct',
                                 '2017-create_date','followers','friends','tweets','verified','avg_retweet','avg_favourite',
                                 'margin_change']]
    summary = pd.concat([pd.DataFrame(df1.count()/len(df1)).T,pd.DataFrame(df1.quantile(0)).T,
                         pd.DataFrame(df1.quantile(0.01)).T,pd.DataFrame(df1.quantile(0.05)).T,pd.DataFrame(df1.quantile(0.1)).T,
                         pd.DataFrame(df1.quantile(0.25)).T,pd.DataFrame(df1.quantile(0.5)).T,pd.DataFrame(df1.quantile(0.75)).T,
                         pd.DataFrame(df1.quantile(0.9)).T,pd.DataFrame(df1.quantile(0.95)).T,pd.DataFrame(df1.quantile(0.99)).T,
                         pd.DataFrame(df1.quantile(1)).T,pd.DataFrame(df1.mean()).T,pd.DataFrame(df1.quantile(0.75)-df1.quantile(0.25)).T,
                         pd.DataFrame(df1.skew()).T,pd.DataFrame(df1.kurt()).T,pd.DataFrame((df1 == 0).astype(int).sum(axis=0)).T,
                         pd.DataFrame((df1>0).astype(int).sum(axis=0)).T,pd.DataFrame((df1<0).astype(int).sum(axis=0)).T,
                         pd.DataFrame(len(df1)-df1.count()).T])
    summary_colunms = ['coverage','mean','std','min','Q1','median','Q3','max','P1','P5','P10','P90','P95','P99','skew','kurt','zero','positive','negative','missing']
    pd.DataFrame(summary).T.to_excel(writer,f"{i}")
writer.save()
writer.close()

In [7]:
def logistic(df1):
    results = dict()
    for i in cols:
        data = df1[[i, 'margin_change']]
        results[i] = dict()
        # sample split
        train, test = train_test_split(data, test_size = 0.3, train_size = 0.7)
        x_train = train.iloc[0:, 0:1]
        y_train = train[['margin_change']]
        x_test = test.iloc[0:, 0:1]
        y_test = test[['margin_change']]
        # fit and predict
        model = linear_model.LogisticRegression(class_weight="balanced")
        model.fit(x_train, y_train)
        predictions = model.predict(x_test)
        actuals = test['margin_change']
        # accuracy
        acc = model.score(x_test,actuals)
        results[i]['accuracy'] = acc
        # gini
        fpr, tpr, thresholds = roc_curve(actuals, predictions)
        roc_auc = auc(fpr, tpr)
        gini = (2 * roc_auc) - 1
        if gini < 0:
            predictions = 1 - predictions
            fpr, tpr, thresholds = roc_curve(actuals, predictions)
            roc_auc = auc(fpr, tpr)
            gini = (2 * roc_auc) - 1
        results[i]['gini'] = gini
        ks_result = ks_2samp(actuals, predictions)
        results[i]['ks_stats'] = ks_result[0]
        results[i]['ks_pvalue'] = ks_result[1] 
    return results

In [11]:
sectors = list(set(df['Sector']))[1:]
#writer = pd.ExcelWriter("lr1.xlsx",engine='openpyxl')
cols = ['dummy_1','dummy_2','dummy_3','dummy_4','dummy_5','dummy_6','media_pct']
names = ['Name','dummy_1','dummy_2','dummy_3','dummy_4','dummy_5','dummy_6','media_pct','margin_change']
df0 = pd.DataFrame([[0,0,0,0,0,0,0,0,0]],columns=names)
for i in sectors:
    try:
        df1 = df[df['Sector'] == i][['dummy_1','dummy_2','dummy_3','dummy_4','dummy_5','dummy_6','media_pct','margin_change']]
        for c in df1.index:
            if df1['margin_change'][c] >= df1['margin_change'].quantile(0.95):
                df1['margin_change'][c] = 1
            else:
                df1['margin_change'][c] = 0
        #results = logistic(df1)
        #pd.DataFrame(results).T.to_excel(writer,f"{i}")
        df0 = pd.concat([df0,df1])
    except:
        pass
#writer.save()
#writer.close()
result = logistic(df0)
pd.DataFrame(result).T.to_excel("lr4.xlsx")

E:\Python\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
E:\Python\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
E:\Python\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()
E:\Python\lib\site-packages\sklearn\linear_mod

In [6]:
sectors = list(set(df['Sector']))[1:]
writer = pd.ExcelWriter("sector.xlsx",engine='openpyxl')
cols = ['2017-create_date','followers','friends','tweets','verified','avg_retweet','avg_favourite']
names = ['Name','2017-create_date','followers','friends','tweets','verified','avg_retweet','avg_favourite','margin_change']
df0 = pd.DataFrame([[0,0,0,0,0,0,0,0,0]],columns=names)
for i in sectors:
    df1 = df[df['Sector'] == i][['Name','2017-create_date','followers','friends','tweets','verified','avg_retweet','avg_favourite',
                                 'margin_change']]
    for a in cols:
        q1 = df1[a].quantile(0.25)
        p90 = df1[a].quantile(0.9)
        for b in df1.index:
            if pd.isnull(df1[a][b]):
                df1[a][b] = 0
            elif df1[a][b] < q1:
                df1[a][b] = 1
            elif df1[a][b] < p90 and df1[a][b] >= q1:
                df1[a][b] = 2
            elif df1[a][b] >= p90:
                df1[a][b] = 3
            else:
                continue
    p95 = df1['margin_change'].quantile(0.95)
    for c in df1.index:
        if df1['margin_change'][c] >= p95:
            df1['margin_change'][c] = 1
        else:
            df1['margin_change'][c] = 0
        #results = logistic(df1)
    pd.DataFrame(df1).to_excel(writer,f"{i}")  
    df0 = pd.concat([df0,df1])
writer.save()
writer.close()
pd.DataFrame(df0).to_excel("part2.xlsx")
#result = logistic(df0)
#pd.DataFrame(result).T.to_excel("lr3.xlsx")

E:\Python\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\Python\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
E:\Python\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\Python\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in 

In [8]:
df0

,Name,2017-create_date,followers,friends,tweets,verified,avg_retweet,avg_favourite,margin_change
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AMERICAN AIRLINES GROUP INC,2.0,3.0,3.0,3.0,3.0,1.0,1.0,0.0
6,AARONS INC,2.0,3.0,2.0,2.0,3.0,2.0,2.0,0.0
8,AAON INC,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0
13,ATLAS AIR WORLDWIDE HOLDINGS INC,2.0,2.0,2.0,1.0,2.0,2.0,2.0,0.0
14,AXON ENTERPRISE INC,2.0,2.0,3.0,2.0,2.0,2.0,3.0,0.0
16,ABB LTD ADR EACH REPR 1 ORD CHF1.03,2.0,3.0,2.0,2.0,3.0,3.0,3.0,0.0
25,ABM INDUSTRIES INC,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0
32,ARCOSA INC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,ACAMAR PARTNERS ACQUISITION CORP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df1 = df[['2017-create_date','followers','friends','tweets','verified','avg_retweet','avg_favourite','margin_change']]
cols = ['2017-create_date','followers','friends','tweets','verified','avg_retweet','avg_favourite']
for a in cols:
    for b in df1.index:
        if pd.isnull(df1[a][b]):
            df1[a][b] = 0
        elif df1[a][b] < df1[a].quantile(0.25):
            df1[a][b] = 1
        elif df1[a][b] < df1[a].quantile(0.9) and df1[a][b] >= df1[a].quantile(0.25):
            df1[a][b] = 2
        elif df1[a][b] >= df1[a].quantile(0.9):
            df1[a][b] = 3
        else:
            continue
    rate_1 = df1[a][df1[a] == 1].count()/len(df1[a])
    rate_2 = df1[a][df1[a] == 2].count()/len(df1[a])
    rate_3 = df1[a][df1[a] == 3].count()/len(df1[a])
    rate_0 = df1[a][df1[a] == 0].count()/len(df1[a])
    rate.append((a,rate_1,rate_2,rate_3,rate_0))
pd.DataFrame(rate).T.to_excel("total.xlsx")

E:\Python\lib\site-packages\IPython\core\interactiveshell.py:3325: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [208]:
df1 = df[['Name','dummy_1','dummy_2','dummy_3','dummy_4','dummy_5','dummy_6','media_pct',
                                 '2017-create_date','followers','friends','tweets','verified','avg_retweet','avg_favourite']]
cols = ['dummy_1','dummy_2','dummy_3','dummy_4','dummy_5','dummy_6','media_pct',
                                 '2017-create_date','followers','friends','tweets','verified','avg_retweet','avg_favourite']
for a in cols:
    for b in df1.index:
        if df1[a][b] > df1[a].quantile(0.99):
            df1[a][b] = df1[a].quantile(0.99)
        elif df1[a][b] < df1[a].quantile(0.01) or pd.isnull(df1[a][b]):
            df1[a][b] = df1[a].quantile(0.01)
        else:
            continue
    print(a)

dummy_1
dummy_2
dummy_3
dummy_4
dummy_5
dummy_6
media_pct


E:\Python\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
E:\Python\lib\site-packages\IPython\core\interactiveshell.py:3325: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
E:\Python\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


2017-create_date
followers
friends
tweets
verified
avg_retweet
avg_favourite


In [209]:
df1.to_csv('data_processed.csv')

In [210]:
df2 = pd.read_csv("processed data.csv")

In [212]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5491 entries, 0 to 5490
Data columns (total 80 columns):
Unnamed: 0                               5491 non-null object
ticker                                   5444 non-null object
Name                                     5444 non-null object
Website                                  5443 non-null object
Sector                                   5436 non-null object
Industry                                 5436 non-null object
Description                              5444 non-null object
Key_Words                                5442 non-null object
Facebook                                 2822 non-null object
Twitter                                  3156 non-null object
Linkedin                                 3120 non-null object
Youtube                                  2119 non-null object
Instagram                                1358 non-null object
wikipedia                                1951 non-null object
dummy_1                  

In [8]:
df = pd.read_excel("processed_data(1).xlsx")

In [9]:
# WOE

import pandas as pd
import numpy as np
import datetime
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve,auc
from scipy.stats import ks_2samp
import math

,Unnamed: 0,ticker,Name,Website,Sector,Industry,Description,Key_Words,Facebook,Twitter,...,total_title,date_2018,date_2017,revenue_2018,revenue_2017,opincome_2018,opincome_2017,margin_2018,margin_2017,margin_change
0,0,A,AGILENT TECHNOLOGIES INC,http://www.agilent.com,Healthcare,Diagnostics & Research,"Agilent Technologies, Inc. provides applicatio...","spectrometry, applied, laboratory, gene, cell",https://www.facebook.com/Agilent.Tech,https://twitter.com/agilent,...,"CEO, Pres & Director ; Sr. VP & CFO ; Sr. VP, ...",2018-10-30,2017-10-30 00:00:00,4914000.0,4472000.0,928000.0,841000.0,0.188848,0.188059,0.004196
1,1,AA,ALCOA CORPORATION,http://www.alcoa.com,Basic Materials,Aluminum,"Alcoa Corporation produces and sells bauxite, ...","aluminum, bauxite, energy, production, headqua...",http://www.facebook.com/alcoa,http://www.twitter.com/alcoa,...,"Pres, CEO & Director ; Exec. VP & CFO ; Exec. ...",2018-12-30,2017-12-30 00:00:00,13403000.0,11652000.0,2310000.0,1514000.0,0.172349,0.129935,0.326431
2,2,AACG,ATA CREATIVITY GLOBAL SPON ADS EACH REP 2 ORD SHS,http://www.atai.net.cn,Consumer Defensive,Education & Training Services,ATA Creativity Global provides educational ser...,"education, programs, educational, changed, net...",NaN,NaN,...,"Chief Financial Officer ; Co-Founder, Chairman...",2018-12-30,2017-03-30 00:00:00,1339.0,472386.0,-67792.0,71807.0,-50.628827,0.152009,-334.064315
3,3,AAL,AMERICAN AIRLINES GROUP INC,http://www.aa.com,Industrials,Airlines,"American Airlines Group Inc., through its subs...","air, changed, transportation, principally, app...",http://facebook.com/aa,http://twitter.com/americanair,...,Chairman & CEO ; Pres ; Exec. VP & CFO ; Exec....,2018-12-30,2017-12-30 00:00:00,42207000.0,40180000.0,4770000.0,5993000.0,0.113014,0.149154,-0.242296
4,4,AAMC,ALTISOURCE ASSET MANAGEMENT CORP,http://www.altisourceamc.com,Financial Services,Asset Management,"Altisource Asset Management Corporation, an as...","agreement, investors, acquires, newsource, fou...",NaN,NaN,...,Chairman & Co-CEO ; Chief Financial Officer ; ...,2018-12-30,2017-12-30 00:00:00,15926.0,18160.0,-6608.0,-7347.0,-0.414919,-0.404570,0.025579
5,5,AAME,ATLANTIC AMERICAN CORP,http://www.atlam.com,Financial Services,Insurance-Life,"Atlantic American Corporation, through its sub...","insurance, coverage, life, casualty, property",NaN,NaN,...,"Chairman, Pres & CEO ; VP, CFO & Corp. Sec. ; ...",2018-12-30,2017-12-30 00:00:00,185553.0,181114.0,NaN,NaN,NaN,NaN,NaN
6,6,AAN,AARONS INC,http://www.aarons.com,Industrials,Rental & Leasing Services,"Aaron's, Inc. operates as an omnichannel provi...","operates, founded, omnichannel, provider, fran...",https://www.facebook.com/aaronsinc/,https://twitter.com/aaronsinc,...,"Pres, CEO & Director ; CFO & Pres of Strategic...",2018-12-30,2017-12-30 00:00:00,3383708.0,3207716.0,588192.0,518885.0,0.173831,0.161762,0.074610
7,7,AAOI,APPLIED OPTOELECTRONICS INC,http://www.ao-inc.com,Technology,Semiconductors,"Applied Optoelectronics, Inc. designs, manufac...","internet, indirect, television, distribution, ...",https://www.facebook.com/Applied.Optoelectroni...,https://twitter.com/AppliedOptoAAOI,...,"Founder, Chairman, Pres & CEO ; CFO & Chief St...",2018-12-30,2017-12-30 00:00:00,267465.0,382329.0,-10768.0,86951.0,-0.040259,0.227425,-1.177023
8,8,AAON,AAON INC,http://www.aaon.com,Industrials,Building Products & Equipment,"AAON, Inc., together with its subsidiaries, en...","air, engages, heating, energy, rooftop",https://www.facebook.com/AAONInc/,https://twitter.com/AAONHVAC,...,"Founder, Chairman & CEO ; Pres & Director ; VP...",2018-12-30,2017-12-30 00:00:00,433947.0,405232.0,55778.0,74148.0,0.128536,0.182977,-0.297526
9,9,AAP,ADVANCE AUTO PARTS INC,http://www.advanceautoparts.com,Consumer Cyclical,Specialty Retail,"Advance Auto Parts, Inc. provides automotive r...","battery, engine, air, oil, stores",http://www.facebook.com/advanceautoparts,http://twitter.com/AdvanceAuto,...,"Pres, CEO & Director ; Exec. VP & CFO ; Exec. ..

In [ ]:
df = pd.read_excel("tree_data_0402.xlsx")
df = df[(df['Sector'] != 'Financial') & (df['Sector'] != 'Industrial Goods') & (df['Sector'] != 'Services') & pd.notnull(df['Sector']) & pd.notnull(df['margin_change_90'])]
cols = ['2017-create_date','followers','friends','tweets','verified','avg_retweet','avg_favourite']
x = df.copy()
for s in set(list(df['Sector'])):
    for a in cols:
        rate = []
        for i in range(0,4):
            non_event = x.loc[:,a][(x.loc[:,a] == i) & (x.loc[:,'margin_change_90'] == 0) & (x.loc[:,'Sector'] == s)].count() 
            event = x.loc[:,a][(x.loc[:,a] == i) & (x.loc[:,'margin_change_90'] == 1) & (x.loc[:,'Sector'] == s)].count() 
            try:
                WOE = float(math.log(non_event/event))
            except:
                WOE = 0
            if (math.isinf(WOE) or math.isnan(WOE)):
                WOE= 0
            rate.append(WOE)
        for b in x.index:
            if x.loc[b,'Sector'] == s:
                if x.loc[b,a] == 0:
                    x.loc[b,a] = rate[0]
                if x.loc[b,a] == 1:
                    x.loc[b,a] = rate[1]
                if x.loc[b,a] == 2:
                    x.loc[b,a] = rate[2]
                if x.loc[b,a] == 3:
                    x.loc[b,a] = rate[3]
x.to_excel("data_0410.xlsx")